# Vision app

In [3]:
%matplotlib widget
from dorna_vision import Detection_app
x = Detection_app()

KeyboardInterrupt: 

# Eye in hand calibration

In [ ]:
from dorna2 import Dorna
from camera import Camera
from dorna_vision import Detection
import time
import json
import config
#####################
import config

pxl_list = [[469, 203]]
pxl = pxl_list[0]
rvec_base = [180, 0, 0]
ip = config.robot["ip"]
joint_img = config.tube_pick["joint"]+config.tube_pick["aux"]
tool = config.two_finger_gripper["15ml_cap"]["tool"]
preset = {
        "camera_mount":{
            "type": "dorna_ta_j4",
            "ej": [0 ,0, 0, 0, 0, 0, 0, 0],
            "T": [46.5174596+1+1+0+4-(5), 32.0776662-3+1-0-1.5+(-3), -4.24772615-3, -0.27547989, 0.27691881, 89.6939516],
        },
}
sim=0
speed = 0.05
###### initialization
# robot
robot = Dorna()
robot.connect(ip)

# camera
camera = Camera()
camera.connect()
Detection(robot=robot, camera=camera, **config.detection_preset["tube"])
# detection
d = Detection(robot=robot, camera=camera, **preset)
######
# go imaging
robot.go(joint=joint_img)
time.sleep(0.5)

# run detection
d.run()

# get xyz
xyz = d.xyz(pxl)
xyz[2] +=2

# go xyz
tvec = xyz+rvec_base
retval = robot.go(motion="lmove", pose=tvec, tool=tool, speed=speed, sim=sim)
for r in retval:
    print(json.dumps(r))

robot.close()
camera.close()
d.close()

{"cmd": "lmove", "rel": 0, "accel": 300.0, "jerk": 500.0, "cont": 0, "corner": 50, "j0": 81.33915294859935, "j1": 35.586706783140656, "j2": -105.16652829962112, "j3": -7.958078640513122e-13, "j4": -20.42017848351952, "j5": 81.33915294860009}


# Calibration with joints

In [1]:
from dorna2 import Dorna
from dorna2 import pose as dorna_pose
import numpy as np
import config

# rail config
frame_in_world = config.robot["frame_in_world"]
base_in_world = config.robot["base_in_world"]
aux_dir = config.robot["aux_dir"]

def update_robot_frame(config):
    config.robot["robot_x_world_off"] = config.robot["rail_screw_pos"] + config.robot["rail_screw_diameter"]/2 - config.robot["rail_scew_dist"] + config.robot["robot_base_x"] + config.robot["rail_x_ref"]
    config.robot["base_in_world"][0] = config.robot["robot_x_world_off"]

update_robot_frame(config=config)

# measured
m_joint_rec = {"cmd":"jmove","rel":0,"j0":-37.55127,"j1":-16.721191,"j2":-56.074219,"j3":0,"j4":-17.20459,"j5":-37.55127,"j6":485}
m_aux = config.cap_holder["aux"]

# target
target_aux = config.cap_holder["aux"]
target_frame = config.cap_holder["frame"]
tool = config.robot_gripper["cap_decap"]
target_pose = config.cap_holder["pick"]

####### measured
m_joint = [m_joint_rec[k] for k in ["j"+str(i) for i in range(6)]]
robot = Dorna()
robot.kinematic.set_tcp_xyzabc(tool)
robot.kinematic.fw(m_joint)
m_xyzabc = robot.kinematic.fw(m_joint)
m_xyzabc_world = dorna_pose.robot_to_frame(robot.kinematic.fw(m_joint), aux=m_aux, aux_dir=aux_dir, base_in_world=base_in_world, frame_in_world = frame_in_world)
print(m_xyzabc_world)

####### object
target_xyzabc_world = dorna_pose.transform_pose(target_pose, from_frame=target_frame, to_frame=[0,0,0,0,0,0])
print(target_xyzabc_world)

####### recalculate o_frame
# ── 2) Build 4×4 transforms for world and local poses ────────────────────
T_world = np.array(dorna_pose.xyzabc_to_T(m_xyzabc_world))
T_local = np.array(dorna_pose.xyzabc_to_T(target_pose))

# ── 3) Solve for the object frame:  T_frame = T_world @ inv(T_local) ─────
T_frame = T_world @ np.linalg.inv(T_local)
target_frame_np = dorna_pose.T_to_xyzabc(T_frame)
target_frame_new = [float(x) for x in target_frame_np]
print("####### target_frame_new ###########")
print(target_frame_new)

#### recalculate target_pose
target_pose_new = list(map( float, dorna_pose.transform_pose( m_xyzabc_world, from_frame=[0, 0, 0, 0, 0, 0], to_frame=target_frame)))
print("####### target_pose_new ###########")
print(target_pose_new)

[935.4041921014215, -165.4618301041079, 3.7603345785304043, 180.0, 0.0, 0.0]
[937.5, -162.5, 0.0, 180.0, 0.0, 0.0]
####### target_frame_new ###########
[935.4041921014215, -165.4618301041079, 3.7603345785304043, 180.0, 0.0, 0.0]
####### target_pose_new ###########
[-2.095807898578528, 2.9618301041078894, -3.760334578530404, 0.0, 0.0, 0.0]


# Rotation

In [7]:
from dorna2 import pose

abc = [127.279221,-127.279221,0]

# abc = pose.rotate_abc(abc, axis=[1, 0, 0], angle=180, local=True)
abc = pose.rotate_abc(abc, axis=[1, 0, 0], angle=-66, local=True)
abc

[86.55613616363203, -86.55613578453197, -56.21021181630984]

# Go

In [ ]:
from dorna2 import Dorna
import time
freedom = {"num":50, "range":[0.5, 0.5, 0.5], "early_exit":True}
robot = Dorna()
current_joint =  [80.112305, 68.862305, -117.685547, -0.021973, -41.176758, 35.134277, 388.74375]
pose =  [-300.4584516808835, 186.73230510526315, 250.1425720414547, -159.00911604446466, 65.86373240657448, -23.19692136401907, 390.35]

final_joint = [141.745605, 48.88916, -80.002441, -10.700684, -72.355957, -168.815918, 390.35625]


#robot.kinematic.inv(pose[0:6], current_joint[0:6], False, freedom=freedom)
robot.go(pose=pose, current_joint=current_joint, motion="lmove", freedom= freedom, cont=0, corner=50, timeout=-1, sim=1)   


[{'cmd': 'lmove',
  'rel': 0,
  'accel': 1200.0,
  'jerk': 2000.0,
  'cont': 0,
  'corner': 50,
  'j0': 141.7349335654358,
  'j1': 48.88754038426882,
  'j2': -79.98482008833054,
  'j3': -10.698901419291417,
  'j4': -72.36859331669143,
  'j5': -168.8100032051365,
  'j6': 390.35}]